# Chat Completion API

In [ ]:
!pip install openai

## 환경설정
1. openai 사이트에서 API KEY 발급
2. google colab - secret탭에 `OPEN_AI_KEY` 등록

In [1]:
from google.colab import userdata
OPENAI_API_KEY = userdata.get('OPENAI_API_KEY')

In [2]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY)

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": "너는 아주 친절하고,많은 도움을 주는 챗봇이야~"
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": "안녕! 내 이름은 한성규야! 지금 너무 더워서 졸려 어떡하지...? "
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

In [ ]:
# ChatCompletion(choices=[Choice(message(content))])
response.choices[0].message.content

'안녕, 한성규야! 더운 날씨 때문에 졸리면 정말 힘들지. 몇 가지 방법을 추천해줄게:\n\n1. **시원한 음료수 마시기**: 찬물이나 시원한 음료를 마시면 기분이 나아질 수 있어.\n2. **가벼운 스트레칭**: 몸을 가볍게 움직이면 피로가 조금 덜해질 수 있어.\n3. **외부 공기 쐬기**: 바람이 시원한 곳으로 나가서 잠깐 신선한 공기를 마셔보는 것도 좋아.\n4. **짧은 낮잠**: 시간이 괜찮다면 10~20분 정도 짧게 눈을 감는 것도 도움이 될 수 있어.\n\n더위에 너무 지치지 않게 조심해! 다른 궁금한 게 있으면 언제든지 말해줘~'

## 프롬프팅의 기본구성

https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

1. Instruction 지시사항
2. Context 문맥
3. Input Data/Example 입력/예시
4. Output Indicator 출력지시

## 기사 제목 교정

- 기자들이 송고한 시가에서 제목을 추출하고, 표현 조정
- 프랑스AFP 속보시스템에서 도입되어 사용

In [ ]:
title_before = '테이의 FM 라디오방송에 주목해주세요'

# 역할 / 페르소나 / 지시사항
system_message = """
기사들이 송고한 제목에서 맞춬법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

- 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
- 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
- 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
- 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

### Steps ###
1. 기사제목을 읽고 주요내용을 이해하세요
2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈 수 있는지 점검하고 간결하게 정리하세요.

### Output Format ###
기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

- 원래 제목: [기사 원래 제목]
- 교정 제목: [교정 기사 제목]

### Examples ###
- 원래 제목: "어제 거울에서 큰 불이 나 수백명이 대피했다."
- 교정 제목: "서울 대형 화재, 수백명 대피"

### Extra Instruction ###
- 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
- 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
- 제목이 특정 집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

"""

user_message = f"""
다음 기사제목을 간결히 수정해주세요.

{title_before}
"""

response = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {
      "role": "system",
      "content": [
        {
          "type": "text",
          "text": system_message
        }
      ]
    },
    {
      "role": "user",
      "content": [
        {
          "type": "text",
          "text": user_message
        }
      ]
    }
  ],
  response_format={
    "type": "text"
  },
  temperature=1,
  max_completion_tokens=2048,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)
print(response.choices[0].message.content)

- 원래 제목: "테이의 FM 라디오방송에 주목해주세요"
- 교정 제목: "테이의 FM 라디오 방송 주목"


In [ ]:
# 함수 리팩토링

def correct_news_title(title_before, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    기사들이 송고한 제목에서 맞춤법, 문법, 의미, 어조등에 있어서 교정작업을 수행해 주세요.

    - 기사 제목이 명확하고 주제와 잘 맞도록 조정하세요.
    - 독자의 관심을 끌 수 있도록 간결하면서도 임팩트 있는 표현을 사용하세요.
    - 어조가 지나치게 감정적이거나 부정적인 경우 표현을 완화하거나, 중립적인 어조로 수정하세요.
    - 비속어가 포함되어 있는 경우, 비속어를 반드시 제거하고, 의미를 적절히 유지하도록 제목을 교정하세요.

    ### Steps ###
    1. 기사제목을 읽고 주요내용을 이해하세요.
    2. 제목이 전달하고자하는 메세지를 명확하게 반영하는지 검토하세요.
    3. 맞춤법, 문법, 의미 전달의 정확성등을 점검하고 적절히 수정하세요.
    4. 제목이 자연스럽고, 독자에게 매력적으로 다가갈수 있는지 점검하고 매우 간결하게 정리하세요.


    ### Output Format ###
    기사 원래 제목과 교정된 제목을 다음 형식으로 제공하세요.

    - 원래 제목: [기사 원래 제목]
    - 교정 제목: [교정 기사 제목]

    ### Examples ###
    - 원래 제목: "어제 서울에서 큰 불이 나 수백명이 대피했다."
    - 교정 제목: "서울 대형 화재, 수백명 대피"

    - 원래 제목: "전기자동차 판매량 급감에 내연차회사들이 즐거워하는 중입니다."
    - 교정 제목: "전기차 판매량 급감에 웃는 내연차회사들"

    ### Extra Instructions ###
    - 제목이 너무 길면, 간결하게 줄이되, 핵심 메세지를 잃어버려서는 안됩니다.
    - 지역명, 시간 등의 중요한 정보는 명확하게 유지하세요.
    - 제목이 특정집단이나 대상에 대해 중립적이지 않을 경우, 그 표현을 완화하세요.

    """

    user_message = f"""
    다음 기사제목을 교정해주세요.

    제목: {title_before}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content

title_before = '주말 미친 폭우 예상. 모두들 무사하시길~'
output = correct_news_title(title_before)
print(output)

- 원래 제목: 주말 미친 폭우 예상. 모두들 무사하시길~
- 교정 제목: "주말 집중호우 예보, 안전에 유의하세요"


## 연애코치 ReAct

In [3]:
def dating_coach(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    어떤 상황에서든 최고의 논리적/감성적 관점을 적용하는 연애코치로써 사용자의 고민을 해결해주세요.

    << Output Format >>
    1. 상황분석:

    2. 행동계획:

    3. 실행:
    """

    user_message = f"""
    사용자 현재상황:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [4]:
dating_coach('저 이번주말에 여자친구와 100일인데. 어떡하죠?')

'1. 상황분석:\n   - 당신과 여자친구의 100일 기념일이 다가오고 있습니다. 이는 두 사람의 관계에서 특별한 의미를 가지며, 기념일을 잘 보내는 것은 서로에 대한 사랑과 소중함을 다시 확인하는 좋은 기회입니다.\n   - 현재 상황에서는 어떤 특별한 계획이나 아이디어가 필요할 것 같습니다. 그녀가 좋아할 만한 활동이나 선물, 혹은 기념일을 특별하게 만들 수 있는 방법을 고민해보는 것이 중요합니다.\n\n2. 행동계획:\n   - 여자친구의 취향과 좋아하는 것들을 바탕으로 기념일을 어떻게 보낼지 계획합니다.\n   - 선물 아이디어: 개인적인 의미가 담긴 물건, 데이트 관련 상품권, 또는 손편지 등을 고려해보세요.\n   - 특별한 데이트 플랜을 세워보세요: 평소 가고 싶은 레스토랑 예약, 피크닉, 영화 감상, 직접 요리해주는 저녁 등.\n   - 기념일에만 사용할 수 있는 특별한 요소를 추가하여 더욱 기억에 남는 날로 만들어주세요.\n\n3. 실행:\n   - 여자친구의 좋아하는 장소나 마음에 드는 액티비티를 선정합니다.\n   - 선물 준비에 착수합니다: 그녀가 갖고 싶어하던 아이템이나 의미 있는 것을 준비하세요. 당신의 마음을 담은 손편지를 추가하는 것도 좋습니다.\n   - 데이트 코스나 세부 계획을 더 구체화하고, 가능하다면 사전 예약을 해두어 원활하게 진행될 수 있도록 합니다.\n   - 기념일 그 날, 모든 준비가 마무리되었을 때, 진심 어린 마음으로 축하해 주며 특별한 행복한 순간들을 함께 나누세요.'

## 냉털마스터 ReAct
- 사용자는 냉장고에 남아있는 음식재료를 알려주면, LLM은 이를 바탕으로 어떤 음식을 만들지를 조언해준다.
- Reasoning/Action을 끌어낼수 있는 적절한 프롬프팅을 작성하세요.

In [24]:
def refrigerator_out(prompt, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    << System Instruction >>
    사용자가 냉장고에 남아있는 음식재료를 알려주면. 창의적이고 어디서도 못본 음식을 만들지 알려주세요.
    칼로리가 낮고 단백질이 높은 메뉴 위주로 알려주세요.
    조리방법이 간단하거나 소요시간이 적게 걸리면 좋을것같습니다.
    1인분기준으로 만들수있게 고려해주세요.

    << Output Format >>
    1. 추천메뉴와 필요한 재료:

    2. 조리방법:

    3. 추천정도:

    4. 소요시간, 영양정보:
    """

    user_message = f"""
    남아있는 음식재료:
    {prompt}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [28]:
print(refrigerator_out('토마토, 된장, 칠면조, 김치, 고수, 낫또, 파스타, 라면수프'))

1. 추천메뉴와 필요한 재료:
   - 메뉴: 칠면조와 토마토 된장 파스타 
   - 필요한 재료: 토마토 1개, 된장 1 큰술, 칠면조 약 100g, 파스타 70g, 고수 약간, (선택) 김치 약간

2. 조리방법:
   1. 끓는 물에 소금을 넣고 파스타를 포장지의 지시에 따라 삶습니다.
   2. 삶은 파스타는 체에 받쳐 물기를 제거합니다.
   3. 다른 팬에 칠면조를 잘게 썰어 중불에서 익힐 때까지 볶습니다.
   4. 익힌 칠면조에 적당한 크기로 자른 토마토를 넣고 볶아주세요.
   5. 된장을 넣고 잘 섞어 맛을 낸 후, 삶은 파스타를 팬에 넣고 고루 버무립니다.
   6. 마지막으로 고수를 뿌리고 필요 시 김치를 곁들이면 완성입니다.

3. 추천정도:
   - 이 요리는 단백질이 풍부하고, 된장의 풍미와 신선한 토마토의 조화가 뛰어납니다. 칠면조 고기를 넣어 더욱 풍부한 식감을 제공하면서도 칼로리는 낮게 유지할 수 있습니다. 독특한 맛의 조화로 누구나 좋아할 만한 메뉴입니다!

4. 소요시간, 영양정보:
   - 소요시간: 약 20 분
   - 영양정보: 한 끼 기준 대략 350 칼로리, 단백질 약 30g, 탄수화물 약 50g, 지방 약 5g(부재료에 따라 달라질 수 있음).


In [29]:
def fridge_raid_master(user_foods, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 사용자 냉장고의 재료를 가지고 최고의 음식을 만들 수 있는 레시피를 추천하는 챗봇입니다.
현재 상황을 분석하고 실행계획을 세우며 추가 내용이 있을지 확인하는 꼼꼼함을 보여주세요.

## 지시사항

### 상황분석
1. **재료 확인**: 냉장고에 있는 재료 목록을 작성하고, 각 재료의 신선함과 사용 가능 시간을 파악합니다.
2. **요리 컨셉 설정**: 재료를 고려하여 요리의 주제를 정하고 잠재적인 요리 아이디어를 계획합니다.

### 실행계획
3. **목표 변경**: 사용 가능한 재료로 만들고자 하는 요리의 목표를 설정하고, 필요한 정보나 추가 자료를 조사합니다.
4. **전략 개발**: 단계별로 목표를 달성하기 위한 전반적인 요리 계획을 세웁니다.
    단, 사용자가 따라하기 쉽게 단계별 가이드를 작성해야 합니다.

### 검증 및 추가내용 확인
5. **레시피 검토**: 작성된 레시피를 검토하여 정보가 명확하고 완전한지 확인합니다.
6. **수정**: 필요 시 수정하여 최종 레시피를 완성합니다.

## 출력형식

1. 상황분석:
   - 재료 목록 및 상태
   - 요리 컨셉 및 아이디어

2. 실행계획:
   - 목표 및 세부 계획
   - 필요한 추가 자료 및 전략

3. 검증 및 추가내용 확인
   - 레시피의 정확성 검토
   - 수정 및 편집

4. 최종레시피
   - 사용자가 따라하기 쉽게 목록으로 작성
   - 필요한 준비물도 작성할것!

## Examples

- **Input**:
    사용자의 냉장고에는 현재 [무, 파, 두부]이/가 있습니다.
-   **Output**:
  1. 상황분석: 오늘은 냉장고에 있는 무, 파, 두부를 이용해 한식 스프를 만들어보겠습니다...
  2. 실행계획: 먼저 무와 파를 얇게 썰어 냄비에 넣고 물을 부어...
  3. 검증 및 추가내용 확인: 레시피를 확인한 결과, 무와 두부의 사용 방법에 대한 설명이 추가로 필요합니다...

## Notes

- 요리의 문화적 관련성과 감수성을 고려합니다.
- 레시피는 예상 독자의 요리 지식 수준에 맞게 조정됩니다.

    """

    user_message = f"""
    사용자의 냉장고에는 현재 {user_foods}이/가 있습니다.
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "text"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [30]:
from IPython.display import Markdown

user_foods = ['소고기', '양파', ' 표고버섯', ' 명란젓', ' 된장']
display(Markdown(fridge_raid_master(user_foods)))

1. **상황분석**:
   - **재료 목록 및 상태**:
     - 소고기: 신선도 양호, 유통기한이 가까운 경우 확인 필요
     - 양파: 신선도 좋음, 유통기한 없음 (채소 특성상)
     - 표고버섯: 신선도 양호, 빨리 소비하는 것이 좋음
     - 명란젓: 유통기한 확인 필요, 보통 냉장보관 시 장기간 보관 가능
     - 된장: 신선도 문제 없음, 장기간 사용 가능
   - **요리 컨셉 및 아이디어**: 
     - 다양한 재료가 조화를 이루는 한식 요리, 예를 들어 '소고기 구이와 표고버섯 볶음' 또는 '된장찌개'를 고려할 수 있습니다. 특히 소고기와 된장을 활용해 구수한 된장찌개를 만드는 것이 좋을 것 같습니다.

2. **실행계획**:
   - **목표 및 세부 계획**: 
     - 목표: 소고기, 표고버섯, 양파를 활용한 된장찌개 만들기
     - 세부 계획:
       1. 소고기를 한입 크기로 썬다.
       2. 양파와 표고버섯을 얇게 썬다.
       3. 냄비에 물을 붓고 끓여서 된장을 풀고, 소고기와 채소를 넣는다.
       4. 중불에서 끓이고, 조리 중간에 맛을 보고 조절한다.
   - **필요한 추가 자료 및 전략**: 
     - 표고버섯과 양파의 조리 시간이 같음을 고려해 적절한 타이밍에 넣는 것이 중요하며, 명란젓은 찌개에 넣지 않고 별도로 반찬으로 제공할 수 있습니다.

3. **검증 및 추가내용 확인**:
   - **레시피의 정확성 검토**: 
     - 재료 및 조리 과정이 이해하기 쉽게 구성되어 있는지 확인
     - 명란젓의 활용 방법 제안
   - **수정 및 편집**: 별도 반찬으로 제공하는 방법에 대해 구체적인 안내 필요.

4. **최종레시피**:
   - **준비물**: 
     - 소고기 200g, 양파 1개, 표고버섯 3~4개, 된장 2큰술, 물 4컵, 소금 및 후추 (취향에 따라)
   - **조리 단계 목록**:
     1. 소고기를 한입 크기로 썰고, 양파는 채 썰기, 표고버섯은 슬라이스한다.
     2. 냄비에 물 4컵을 붓고 끓인다.
     3. 물이 끓으면 된장 2큰술을 넣고 잘 풀어준다.
     4. 소고기를 넣고 중불에서 약 10분 정도 끓인다.
     5. 양파와 표고버섯을 추가하고, 약 5분 더 끓인다.
     6. 간을 보고 소금이나 후추로 기호에 맞게 간을 맞춘다.
     7. 완성된 된장찌개를 그릇에 담고, 명란젓을 별도로 준비하여 반찬으로 제공한다.

이제 준비된 재료로 맛있는 된장찌개를 만들어 보세요! 추가 질문이 있다면 언제든지 말씀해 주세요.

## 면접질문 생성 JSON 출력

In [31]:
def job_interview(job_posting, model='gpt-4.1-nano', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
    당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로서, 해당분야의 으뜸가는 면접관입니다.
    매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.

    << 지시사항 >>
    - 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
    - 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.

    << 출력형식 >>
    출력은 JSON형식으로만 반환되어야 합니다.

    {{
        "hard_skill": [
          {{
            "question":"",
            "answer":"",
          }}
        ],
        "soft_skill": [
          {{
            "question":"",
            "answer":"",
          }}
        ]
    }}

    """

    user_message = f"""
채용공고:
{job_posting}
    """

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content


In [37]:
job_posting = """
| 모집부문                      | 담당업무                                                                                                                                                                                                                                                                                               | 자격요건 및 필수사항                                                                                                                                                                                                                                                                                                                                                                 |
| ------------------------- | -------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- | --------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------- |
| **AI 인프라 구축 (ML/DEVOps)** | 1. AI 모델의 배포를 위한 인프라 구축 및 운영<br>2. 알고리즘 개발자에 대한 기술 지원<br>3. 머신러닝 인프라의 설계 및 개발, 운영<br>4. 모니터링, 로깅 시스템의 개발 및 구현<br>5. 머신러닝 시스템의 학습, 배포, 테스트 환경 구축, 운영 및 자동화<br>6. 오픈소스 기반의 플랫폼 통합 및 내재화 구축<br>(MLflow, LakeFS, CVAT, BentoML 등)<br>7. 보안, 인증, 권한 관리 등 운영 정책 및 관리자 기능 구축<br>8. 시스템과 프로세스에 대한 문서 작업 | **학력**<br>- 대졸이상(4년제)<br><br>**경력**<br>- 신입<br><br>**필수사항**<br>- Python 프로그래밍 능력<br>- 컨테이너 오케스트레이션 툴 활용 능력(Docker, Kubernetes 등)<br>- Jenkins, ArgoCD, Gitlab 등 구축 및 운영 경험<br><br>**우대사항**<br>- DevOps 환경 구축 혹은 DevOps 환경에서 개발 및 배포 경험 보유자<br>- MLflow, Kubeflow, ML 모델 서빙 플랫폼 등과 관련된 실무 경험 보유자<br>- Kubernetes, Docker, GitOps, CI/CD, Vault, MinIO 등 클라우드 네이티브 인프라 경험 보유자 |
"""
output = job_interview(job_posting)
print(type(output))

<class 'str'>


In [40]:
import json

# 응답데이터가 json형식이 아니라면 오류발생
data = json.loads(output)
print(type(data))

hard_skill_qa = data['hard_skill']
soft_skill_qa = data['soft_skill']
print(hard_skill_qa)
print(soft_skill_qa)

<class 'dict'>
[{'question': 'AI 인프라 구축 업무에서 가장 핵심이 되는 기술은 무엇이라고 생각하며, 그 이유는 무엇인가요?', 'answer': 'AI 인프라 구축에서 가장 핵심이 되는 기술은 Docker와 Kubernetes로, 이는 컨테이너 오케스트레이션과 확장성을 보장하여 안정적이고 효율적인 모델 배포 및 운영을 가능하게 하기 때문입니다. 이러한 도구들은 CI/CD 파이프라인과의 연계도 원활하게 하여 자동화와 빠른 배포를 지원합니다.'}, {'question': 'MLflow, Kubeflow 등 오픈소스 플랫폼과 관련된 경험이 있다면 어떤 프로젝트에서 어떻게 활용했는지 구체적으로 설명해 주세요.', 'answer': '저는 이전 프로젝트에서 MLflow를 활용하여 모델 실험과 버전 관리를 수립했고, Kubeflow를 통해 모델 학습 파이프라인을 자동화 했습니다. 예를 들어, MLflow를 통해 다양한 하이퍼파라미터 실험 결과를 관리하고, Kubeflow 파이프라인을 설정하여 지속적 통합 및 배포를 구현했습니다. 이를 통해 개발 속도를 높이고 모델 품질을 유지할 수 있었습니다.'}]
[{'question': '팀 내에서 DevOps 환경 구축 경험이 있는 경우, 어떤 역할을 맡았고 어떤 어려움을 어떻게 해결했는지 설명해 주세요.', 'answer': '저는 DevOps 환경 구축 프로젝트에서 인프라 설계와 자동화 파이프라인을 개발하는 역할을 맡았습니다. 초기에는 서비스 간 연계 문제와 권한 관리의 복잡성으로 어려움이 있었지만, 팀원들과 협력하여 역할 기반 접근제어와 문서화, 그리고 CI/CD와 모니터링 시스템을 체계적으로 구축하여 문제를 해결했습니다. 덕분에 팀 전체의 배포 효율성과 안정성이 크게 향상되었습니다.'}, {'question': '새로운 기술이나 도구를 학습할 때, 어떻게 접근하며 빠르게 습득하는 편인가요?', 'answer': '저는 우선 공식 문서와 튜토리얼을 통해 기본 개념을 이해하고, 작은 프로젝트에 바로 적

In [ ]:
def job_interview(job_posting, model='gpt-4o-mini', temperature=1, top_p=1):
    # 역할/페르소나/지시사항
    system_message = """
당신은 머신러닝/딥러닝/LLM/AI서비스개발의 전문가로써, 해당분야의 으뜸가는 면접관입니다.
매번 그룹사의 인재를 발굴하기 위해 면접질문/모범답안을 작성하고 있습니다.
"""

    user_message = f"""

<<지시사항>>
- 사용자가 제출한 채용공고의 내용을 바탕으로 예상면접질문과 답변을 작성해주세요.
- 하드스킬과 소프트스킬 두개의 섹션으로 나누어 작성해주세요.
- 각 스킬별로 질문/답변을 3개씩 만들어주세요.

<<출력형식>>
출력은 json형식으로만 반환되어야 합니다.

{{
    "hard_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ],
    "soft_skill": [
        {{
            "question": "",
            "answer": "",
        }}
    ]
}}

<< 채용공고 >>
{job_posting}
"""

    response = client.chat.completions.create(
        model=model,
        messages=[
            {
                "role": "system",
                "content": [
                    {
                    "type": "text",
                    "text": system_message
                    }
                ]
            },
            {
                "role": "user",
                "content": [
                    {
                    "type": "text",
                    "text": user_message
                    }
                ]
            }],
            response_format={
                "type": "json_object"
            },
        temperature=temperature,
        max_completion_tokens=2048,
        top_p=top_p,
        frequency_penalty=0,
        presence_penalty=0
    )
    return response.choices[0].message.content
